# 프로젝트 - Movielens 영화 추천 실습

전 스텝에서 배운 MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보자.

이번에 활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터.

* 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장.
* 별점 데이터는 대표적인 explicit 데이터. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있음.
* 별점을 시청횟수로 해석해서 생각.
* 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외.

### 1) 데이터 준비와 전처리
-----
Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있음.

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
# 검색을 쉽게하기 위해 제목 문자열을 소문자로 바꿔줍시다.

movies['title'] = movies['title'].str.lower()
movies.head()

,movie_id,title,genre
0,1,toy story (1995),Animation|Children's|Comedy
1,2,jumanji (1995),Adventure|Children's|Fantasy
2,3,grumpier old men (1995),Comedy|Romance
3,4,waiting to exhale (1995),Comedy|Drama
4,5,father of the bride part ii (1995),Comedy


In [7]:
# timestamp 칼럼을 제거
del ratings['timestamp']

In [8]:
# genre(장르) 칼럼을 제거
del movies['genre']

In [9]:
# ratings = ratings.drop(columns=['timestamp', 'genre'])
# ratings.head()

In [10]:
# ratings 와 movies 데이터 프레임 합치기
ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
ratings.head()

,user_id,movie_id,counts,title
0,1,1193,5,one flew over the cuckoo's nest (1975)
1,1,661,3,james and the giant peach (1996)
2,1,914,3,my fair lady (1964)
3,1,3408,4,erin brockovich (2000)
4,1,2355,5,"bug's life, a (1998)"


In [11]:
# user_id가 92인 사용자가 어떤 영화를 시청했는지 확인.
movie_id_user92 = ratings[ratings['user_id']== 1]['title']
movie_id_user92

0                one flew over the cuckoo's nest (1975)
1                      james and the giant peach (1996)
2                                   my fair lady (1964)
3                                erin brockovich (2000)
4                                  bug's life, a (1998)
5                            princess bride, the (1987)
6                                        ben-hur (1959)
7                             christmas story, a (1983)
8                snow white and the seven dwarfs (1937)
9                              wizard of oz, the (1939)
10                          beauty and the beast (1991)
11                                          gigi (1958)
12                        miracle on 34th street (1947)
13                      ferris bueller's day off (1986)
14                           sound of music, the (1965)
15                                     airplane! (1980)
16                                        tarzan (1999)
17                                         bambi

여기까지가 전처리. 이후에는 이전 스텝에 소개했던 것과 동일한 방식으로 MF model을 구성하여 내가 좋아할 만한 영화를 추천해 볼 수 있음.

### 2) 분석해 봅시다.
-----
* ratings에 있는 유니크한 영화 개수
* ratings에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [12]:
# 영화 개수
ratings['movie_id'].nunique()

3628

In [13]:
# 사용자 수
ratings['user_id'].nunique()

6039

In [14]:
# 인기있는 영화 30
movie_count = ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [15]:
# 유저별 몇 편의 영화를 시청했는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [16]:
# 영화 별 시청(count)횟수 중앙값에 대한 통계
user_median = ratings.groupby('user_id')['counts'].median()
user_median.describe()

count    6039.000000
mean        4.055970
std         0.432143
min         3.000000
25%         4.000000
50%         4.000000
75%         4.000000
max         5.000000
Name: counts, dtype: float64

### 3) 내가 선호하는 영화를 5가지 골라서 ratings에 추가해 줍시다.
-----

In [17]:
# title = movies['title'].values
# title

In [18]:
# movie_id = movies['movie_id'].values
# movie_id

In [19]:
# movie_dic = { movie_id:title for movie_id, title in zip(movie_id, title) }
# movie_dic

In [20]:
# 내가 선호하는 영화 5가지
my_favorite = [648, 2571, 1580, 1240, 3623]

# id 리스트를 title 리스트로 변환.
my_favorite_movie = []
for i in my_favorite:
    my_favorite_movie.extend(list(movies[movies['movie_id']==i]['title']))

# '6041'이라는 'user_id'가 위의 영화를 5회씩 시청했다고 가정.
my_movielist = pd.DataFrame({'user_id': ['6041']*5, 'movie_id': my_favorite, 'counts':[5]*5,  'title': my_favorite_movie})
my_movielist

,user_id,movie_id,counts,title
0,6041,648,5,mission: impossible (1996)
1,6041,2571,5,"matrix, the (1999)"
2,6041,1580,5,men in black (1997)
3,6041,1240,5,"terminator, the (1984)"
4,6041,3623,5,mission: impossible 2 (2000)


In [21]:
if not ratings.isin({'user_id':['6041']})['user_id'].any():
    ratings = ratings.append(my_movielist, ignore_index=True)

ratings.tail(10)       # 잘 추가되었는지 확인.

,user_id,movie_id,counts,title
836473,6040,1090,3,platoon (1986)
836474,6040,1094,5,"crying game, the (1992)"
836475,6040,562,5,welcome to the dollhouse (1995)
836476,6040,1096,4,sophie's choice (1982)
836477,6040,1097,4,e.t. the extra-terrestrial (1982)
836478,6041,648,5,mission: impossible (1996)
836479,6041,2571,5,"matrix, the (1999)"
836480,6041,1580,5,men in black (1997)
836481,6041,1240,5,"terminator, the (1984)"
836482,6041,3623,5,mission: impossible 2 (2000)


### 4) CSR matrix를 직접 만들어 봅시다.
-----

In [22]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 유저, 영화 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [23]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['6041'])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6041이 나와야 함. 
print(movie_to_idx['mission: impossible (1996)'])

6039
58


In [24]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_ratings = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_ratings) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_ratings   # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 movie 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_ratings = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_ratings) == len(ratings):
    print('movie  column indexing OK!!')
    ratings['title'] = temp_movie_ratings
else:
    print('movie column indexing Fail!!')

ratings

user_id column indexing OK!!
movie  column indexing OK!!


,user_id,movie_id,counts,title
0,0,1193,5,0
1,0,661,3,1
2,0,914,3,2
3,0,3408,4,3
4,0,2355,5,4
...,...,...,...,...
836478,6039,648,5,58
836479,6039,2571,5,124
836480,6039,1580,5,175
836481,6039,1240,5,200


#### CSR matrix 생성

In [25]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['title'].nunique()

csr_ratings = csr_matrix((ratings.counts, (ratings.user_id, ratings.title)), shape= (num_user, num_movie))
csr_ratings

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.
-----

In [26]:
#[[ your code ]]

from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [27]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [28]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_ratings_transpose = csr_ratings.T
csr_ratings_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [29]:
# 모델 훈련
als_model.fit(csr_ratings_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

### 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
-----

In [30]:
#[[ your code ]]

bom, mission = user_to_idx['6041'], movie_to_idx['mission: impossible (1996)']

bom_vector, mission_vector = als_model.user_factors[bom], als_model.item_factors[mission]
print('슝=3')

슝=3


In [31]:
bom_vector

array([ 9.60037887e-01,  4.56339955e-01, -6.98009491e-01,  1.32376671e-01,
       -2.90270418e-01, -5.80006123e-01, -2.49372229e-01,  6.33196890e-01,
        8.82005692e-01,  1.72089532e-01,  2.34960362e-01, -5.14051616e-01,
        3.34529310e-01,  3.73436958e-01,  2.34871700e-01, -1.76156506e-01,
        1.17498243e+00,  1.05443716e+00, -4.17632937e-01,  1.02343440e-01,
       -1.28060627e+00,  9.22156945e-02,  3.51641588e-02, -2.92548556e-02,
        5.88888228e-02, -2.38320217e-01, -7.69748747e-01,  1.00630954e-01,
        1.30639106e-01,  1.78310588e-01, -7.62635529e-01,  6.05957806e-01,
        5.47584295e-01, -9.70015228e-02, -5.43254852e-01,  1.32438540e-01,
        2.90463835e-01, -1.67574972e-01,  2.02958077e-01,  1.98900878e-01,
       -7.36391723e-01,  4.90176260e-01,  3.59829187e-01,  8.24575648e-02,
       -8.63540530e-01,  1.27366275e-01, -1.08155712e-01, -4.62950975e-01,
       -2.91099370e-01,  7.48185337e-01, -7.20817864e-01, -1.16275442e+00,
       -5.55196628e-02,  

In [32]:
mission_vector

array([ 1.06239319e-02,  7.01617077e-03, -1.53506296e-02,  1.11652669e-02,
        6.16850797e-03, -9.22922324e-03,  7.60580588e-04,  2.61902455e-02,
        1.09913778e-02, -4.43844218e-03,  8.36829096e-03, -3.26055847e-02,
        6.88389968e-03, -5.40135056e-03,  1.52924955e-02,  2.12032516e-02,
        2.96972580e-02,  4.44300398e-02, -1.67873781e-02,  5.91114163e-03,
       -1.51494576e-03,  1.46993380e-02,  2.04874557e-02,  2.38717850e-02,
        2.99423710e-02,  2.89157387e-02, -2.23791413e-02, -1.43071925e-02,
       -2.21139174e-02,  1.75475497e-02, -2.74364036e-02,  1.66404024e-02,
        4.05239686e-03,  1.23032453e-02, -1.14622554e-02,  8.56767595e-03,
        2.33784448e-02,  1.89617444e-02, -2.88731325e-03,  2.56679729e-02,
       -2.95580812e-02,  4.64753772e-04,  1.16966320e-02,  7.12606311e-03,
       -7.54814409e-03,  1.26753293e-03, -1.16479234e-03,  1.83549765e-02,
        1.11383619e-02,  1.22714285e-02, -9.21402872e-03, -5.53525845e-03,
        3.73121947e-02,  

In [33]:
# bom과 mission을 내적하는 코드 : 내가 좋아하는 영화에 대한 선호도 예측
np.dot(bom_vector, mission_vector)

0.5254852

In [34]:
# 다른 영화에 대한 선호도 예측
bom2 = movie_to_idx['terminator, the (1984)']
bom2_vector = als_model.item_factors[bom2]
np.dot(bom_vector, bom2_vector)

0.42963547

### 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.
-----

In [35]:
#[[ your code ]]

favorite_movie = 'sixth sense, the (1999)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(38, 0.99999994),
 (121, 0.4909378),
 (233, 0.4539584),
 (220, 0.40958),
 (273, 0.39929873),
 (170, 0.39321187),
 (796, 0.38530102),
 (99, 0.3597511),
 (243, 0.35422155),
 (670, 0.34867734),
 (1938, 0.33924192),
 (124, 0.33714926),
 (808, 0.33648202),
 (818, 0.33343652),
 (157, 0.33310473)]

In [36]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성.
idx_to_movie  = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie [i[0]] for i in similar_movie]

['sixth sense, the (1999)',
 'silence of the lambs, the (1991)',
 'usual suspects, the (1995)',
 'seven (se7en) (1995)',
 'fight club (1999)',
 'being john malkovich (1999)',
 'bone collector, the (1999)',
 'american beauty (1999)',
 'ghostbusters (1984)',
 'galaxy quest (1999)',
 'superstar (1999)',
 'matrix, the (1999)',
 'end of days (1999)',
 'stigmata (1999)',
 'shawshank redemption, the (1994)']

In [37]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

print("슝=3")

슝=3


In [38]:
get_similar_movie('sixth sense, the (1999)')

['sixth sense, the (1999)',
 'silence of the lambs, the (1991)',
 'usual suspects, the (1995)',
 'seven (se7en) (1995)',
 'fight club (1999)',
 'being john malkovich (1999)',
 'bone collector, the (1999)',
 'american beauty (1999)',
 'ghostbusters (1984)',
 'galaxy quest (1999)']

### 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.
-----

In [39]:
#[[ your code ]]

user = user_to_idx['6041']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_ratings, N=20, filter_already_liked_items=True)
movie_recommended

[(92, 0.61487),
 (107, 0.526078),
 (372, 0.4771634),
 (62, 0.45017248),
 (67, 0.41191846),
 (746, 0.34975153),
 (44, 0.30248734),
 (64, 0.298372),
 (117, 0.2955354),
 (375, 0.2831174),
 (346, 0.27694067),
 (141, 0.27433467),
 (82, 0.26537085),
 (670, 0.25994635),
 (474, 0.25563854),
 (22, 0.2522382),
 (145, 0.24720839),
 (150, 0.24539125),
 (1537, 0.23843439),
 (60, 0.23647149)]

In [40]:
[idx_to_movie[i[0]] for i in movie_recommended]

['terminator 2: judgment day (1991)',
 'jurassic park (1993)',
 'x-men (2000)',
 'total recall (1990)',
 'gladiator (2000)',
 'perfect storm, the (2000)',
 'star wars: episode iv - a new hope (1977)',
 'star wars: episode vi - return of the jedi (1983)',
 'star wars: episode v - the empire strikes back (1980)',
 'face/off (1997)',
 'patriot, the (2000)',
 'fugitive, the (1993)',
 'lost world: jurassic park, the (1997)',
 'galaxy quest (1999)',
 'chicken run (2000)',
 'back to the future (1985)',
 'fifth element, the (1997)',
 'independence day (id4) (1996)',
 'u-571 (2000)',
 'star wars: episode i - the phantom menace (1999)']

In [41]:
bb = movie_to_idx['alien (1979)']
explain = als_model.explain(user, csr_ratings, itemid=bb)

In [42]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('terminator, the (1984)', 0.21722437757576682),
 ('matrix, the (1999)', 0.07835023864704715),
 ('mission: impossible 2 (2000)', -0.0015444121890415847),
 ('mission: impossible (1996)', -0.019594697348800524),
 ('men in black (1997)', -0.0705254994481379)]

## 회고
지금까지 진행해온 프로젝트 중에 내 기준으로 가장 실용적으로 활용 가능한 프로젝트였다.
처음 추천시스템을 접했을 땐, 자연어처리라고 생각했는데 직접 실습을 해보니 자연어처리와는 또 다른 맥락으로 분류되야 할 것 같았다.

lms에서 실습했던 '비슷한 아티스트 찾기 + 유저에게 추천하기'를 되짚어 가며 프로젝트를 진행했다. 모델을 이해하려고 여러차례 반복적으로 보고 또 보고 수정하고 또 수정했던 것 같다. 결과를 도출해내는 것도 프로젝트를 진행하는 것도 막막하기만 했는데 갑자기 번뜩이면서 수정해야할 부분이 눈에 보이기 시작했다.

데이터 준비와 전처리 과정에서 indexing을 잘못 지정해서 데이터 탐색 부분에서 계속적으로 에러가 나서 프로젝트 진행이 안됬는데, 처음부터 차근차근 변수명과 잘못 지정된 코드들을 찾아내가면서 '에러를 수정해가는 과정에서 배운다는 말이 이거구나'하며 즐겁게 에러를 해결했던 것 같다.

인공지능 수학적 지식이나 이론 부분에서 아직 부족한게 많아 모델이 예측한 결과에 대한 해석에 대해서 완벽하게 습득되지는 않아서 아쉬운 것 같다.


(영화를 즐겨보는 편이 아니여서 모델이 내가 선호하는 영화와 유사한 영화를 추천해 줬는지 판단할 수 없지만, 수많은 영화들 중에서 모델에게 추천받은 영화의 리뷰들은 한번 찾아볼 생각이다.)